In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
n_samples = 4
n_length = 100

x_batch = np.random.normal(size=(n_samples, n_length)).astype(np.float32)
y_batch = np.zeros((n_samples, n_length)).astype(np.int32)
for i in range(n_samples):
    start = np.random.choice(n_length // 2)
    end = start + np.random.choice(n_length // 2)
    if i < n_samples//2:
        y_batch[i, start:end+1] = 1

In [ ]:
for i in range(n_samples):
    fig, ax = plt.subplots(2, 1, sharex=True, figsize=(6, 3), dpi=80)
    ax[0].plot(x_batch[i, ...])
    ax[1].plot(y_batch[i, ...])
    ax[1].set_ylim([-0.1, 1.1])
    plt.show()


In [ ]:
tf.reset_default_graph()
inputs = tf.placeholder(shape=[None, n_length], dtype=tf.float32)
labels = tf.placeholder(shape=[None, n_length], dtype=tf.int32)

outputs = inputs - 1

# weight_negative = 0.5
# weight_positive = 2.0
# class_weights = tf.stack([weight_negative, weight_positive], axis=0)
# weights = tf.gather(class_weights, y)

n_negative = tf.cast(tf.reduce_sum(1 - labels), tf.float32)
n_positive = tf.cast(tf.reduce_sum(labels), tf.float32)
p_drop_negative = (n_negative - n_positive) / n_negative  # in (0, 1)
random_mask = tf.random.uniform(
    tf.shape(outputs),
    minval=0.0,
    maxval=1.0,
    dtype=tf.float32)
random_mask = random_mask + tf.cast(labels, tf.float32)
binarized_mask = tf.cast(tf.math.greater(random_mask, p_drop_negative), tf.float32)

loss_per_sample = binarized_mask

# loss_per_sample = out ** 2 * weights
total_loss = tf.reduce_sum(loss_per_sample) / n_length

sess = tf.Session()
tf.global_variables_initializer()


In [ ]:
np_loss_per_sample = sess.run(loss_per_sample, feed_dict={labels: y_batch, inputs: x_batch})

for i in range(n_samples):
    fig, ax = plt.subplots(3, 1, sharex=True, figsize=(6, 3), dpi=80)
    ax[0].plot(x_batch[i, ...])
    ax[1].plot(y_batch[i, ...])
    ax[1].set_ylim([-0.1, 1.1])
    ax[2].plot(np_loss_per_sample[i, ...], '.')
    plt.show()

In [ ]:
np.sum(np_loss_per_sample * y_batch)

In [ ]:
np.sum(np_loss_per_sample * (1-y_batch))